## Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from imblearn.over_sampling import SMOTE


C:\Users\PC\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Importing Data

In [2]:
df = pd.read_csv("labeled_data.csv")

## data Preprocessing

In [3]:
df.sample(5)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
301,306,3,1,2,0,1,"""@_simplycaitlin: why do my friends hate me"" b..."
5394,5551,3,0,3,0,1,@_cblaze sorry i eat pussy
8107,8332,6,2,1,3,2,Black guy in class: *attempts to throw a paper...
4438,4569,3,2,1,0,0,@Princesslexii16 Fucking coon
21419,21881,3,0,3,0,1,That other shit for the birds . Get over yo self


In [4]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [5]:
df.sample(2)

,count,hate_speech,offensive_language,neither,class,tweet
14784,3,0,3,0,1,RT @CourtneyyKay: Cruising down the street in ...
7549,3,0,3,0,1,Alabama won tho. So roll tide bitch


In [6]:
df.shape

(24783, 6)

In [7]:
df['hate_speech'].value_counts(), df['offensive_language'].value_counts(), df['neither'].value_counts()

(hate_speech
 0    19790
 1     3419
 2     1251
 3      287
 4       21
 5        7
 6        5
 7        3
 Name: count, dtype: int64,
 offensive_language
 3    13383
 2     4246
 0     3475
 1     2066
 6      857
 5      369
 4      251
 9       66
 8       37
 7       33
 Name: count, dtype: int64,
 neither
 0    18892
 3     2790
 1     1694
 2     1200
 6      103
 5       54
 4       35
 9        5
 8        5
 7        5
 Name: count, dtype: int64)

In [8]:
df['class'].value_counts()

class
1    19190
2     4163
0     1430
Name: count, dtype: int64

In [9]:
df['tweet'].values

array(["!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
       '!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
       '!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit',
       ...,
       'young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again',
       'youu got wild bitches tellin you lies',
       '~~Ruffled | Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow &amp; white. A Coll http://t.co/H0dYEBvnZB'],
      dtype=object)

## Text Preprocessing

In [21]:
# Download necessary data for tokenization
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
# Download necessary data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # for better lemmatization support

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [27]:
# # initialize tokenizer and stop words

# lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))


In [28]:
# def text_preprocess(text):
#     # To lower case
#     text = text.lower()

#     # remove punctuations
    
#     # removing rt short for retweet
#     text = re.sub(r'\brt\b', '', text)
#     # Remove usernames (words starting with @)
#     text = re.sub(r'@\w+', '', text)
#     # Remove 'amp' and other isolated HTML artifacts
#     text = re.sub(r'\bamp\b', '', text)
#     # Remove punctuation and special characters
#     text = re.sub(r'[^\w\s]', '', text)
#     text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
#     # tokenize text into words
#     words = word_tokenize(text)

#     # remove stop words
#     filtered_tokens = [word for word in words if word not in stop_words]

#     lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

#     return ' '.join(lemmatized_tokens)




In [29]:
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

In [30]:
def text_preprocess(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove 'rt' (retweet), usernames, 'amp', and punctuation
    text = re.sub(r'\brt\b', '', text)  # Remove 'rt' for retweet
    text = re.sub(r'@\w+', '', text)  # Remove usernames (words starting with @)
    text = re.sub(r'\bamp\b', '', text)  # Remove 'amp' and other HTML artifacts
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters

    # Process the text with spaCy
    doc = nlp(text)

    # Lemmatize and remove stop words using spaCy
    lemmatized_tokens = [token.lemma_ for token in doc if token.text not in stop_words]

    # Join the lemmatized tokens into a single string
    return ' '.join(lemmatized_tokens)

# Example usage
text = "The children are playing and they are running quickly, but the dog has been running for hours, and the men are fishermen who were fishing yesterday."
processed_text = text_preprocess(text)
print(processed_text)

child play run quickly dog run hour man fisherman fish yesterday


In [31]:
# Example usage
text = "The children are playing and they are running quickly, but the dog has been running for hours, and the men are fishermen who were fishing yesterday."
processed_text = text_preprocess(text)
print(processed_text)

child play run quickly dog run hour man fisherman fish yesterday


In [32]:
df['cleaned_tweets'] = df['tweet'].apply(text_preprocess)

In [33]:
df.head(10)

,count,hate_speech,offensive_language,neither,class,tweet,cleaned_tweets
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,woman not complain clean house man alway...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,boy dat coldtyga dwn bad cuffin dat hoe 1s...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,dawg ever fuck bitch start cry confuse ...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,look like tranny
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,shit hear might true might faker bitch tel...
5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",shit blow meclaim faithful somebody still fu...
6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ...",sit hate another bitch get much shit go
7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,8220 cause tired big bitch come we skinny girl...
8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ...",might get ya bitch back
9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria...",hobby include fight mariam \n\n bitch


In [34]:
df['tweet'].values[1]

'!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!'

In [35]:
df['cleaned_tweets'].values[0]

'    woman not complain clean house   man always take trash'

In [36]:
df['cleaned_tweets'].values[1]

'    boy dat coldtyga dwn bad cuffin dat hoe 1st place'

In [37]:
df['class'].value_counts()

class
1    19190
2     4163
0     1430
Name: count, dtype: int64

## Balancing Data

In [85]:
class_1 = df[df['class'] == 1].sample(10000, random_state=42)
class_0 = df[df['class'] == 0]
class_2 = df[df['class'] == 2]

class_1.shape, class_0.shape, class_2.shape

from sklearn.utils import resample

zero_upsampled = resample(class_0, replace=True, n_samples=7000, random_state=42)
two_upsampled = resample(class_2, replace=True, n_samples=7000, random_state=42)

# Combine the oversampled data with the '1' class
balanced_df = pd.concat([class_1, zero_upsampled, two_upsampled])

# Shuffle the dataset to ensure randomness
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

balanced_df.shape





(24000, 7)

In [86]:
balanced_df.head(3)

,count,hate_speech,offensive_language,neither,class,tweet,cleaned_tweets
0,3,0,3,0,1,"That, and we spent half of our junior year and...",spend half junior year senior year bitching du...
1,3,0,0,3,2,RT @JohnBishop100: Top tip of the day - if you...,top tip day want inconspicuous elbow gig ...
2,3,0,0,3,2,@slayerific13 @brenda_skelton @daisymcgarr I d...,not hunt bird unless count shooting dive s...


In [87]:
balanced_df['class'].value_counts()

class
1    10000
2     7000
0     7000
Name: count, dtype: int64

## Splitting Data

In [103]:
# x = df.drop(columns=['class'])

x = balanced_df[['cleaned_tweets', 'hate_speech', 'offensive_language', 'neither']] 
y = balanced_df['class']

In [104]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [105]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((19200, 4), (4800, 4), (19200,), (4800,))

## Applying TF_IDF

In [119]:
tf_idf = TfidfVectorizer()

In [120]:
tf_train_data = tf_idf.fit_transform(x_train['cleaned_tweets'])
tf_test_data = tf_idf.transform(x_test['cleaned_tweets'])


In [121]:
tf_train_data.shape, tf_test_data.shape

((19200, 16539), (4800, 16539))

In [122]:
# tf_train_data.value_counts()

## Balancing data

Using SMOTE

In [123]:
smote = SMOTE(random_state = 42)

In [124]:
x_train_balanced, y_train_balanced = smote.fit_resample(tf_train_data, y_train)

In [125]:
x_train_balanced.shape, y_train_balanced.shape

((23979, 16539), (23979,))

In [126]:
y_train_balanced.value_counts()

class
0    7993
1    7993
2    7993
Name: count, dtype: int64

## Building Model

## Random Forest

In [127]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, classification_report

In [128]:
# Train Random Forest model
ranfor_model = RandomForestClassifier(n_estimators=70, random_state=42)
ranfor_model.fit(x_train_balanced, y_train_balanced)

# Evaluate the model
y_pred2 = ranfor_model.predict(tf_test_data)
print("Accuracy:", accuracy_score(y_test, y_pred2))
print("Classification Report:")
print(classification_report(y_test, y_pred2))

Accuracy: 0.9508333333333333
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      1401
           1       0.97      0.91      0.94      2007
           2       0.94      0.97      0.96      1392

    accuracy                           0.95      4800
   macro avg       0.95      0.96      0.95      4800
weighted avg       0.95      0.95      0.95      4800



## Testing with some examples

In [133]:
# Example new tweets for testing
new_tweets = [
    "1) I hate this place, it's full of rude people! #frustrated (Offensive)",  # Offensive Language
    "2) This is an amazing movie! I loved every second of it! 😍 (Neither)",  # Neither (Positive, neutral)
    "3) Why is everyone so mean on the internet these days? 😡 (Offensive)",  # Offensive Language (Expressing frustration towards people being mean)
    "4) I love learning new things, it's such a great experience. (Neither)",  # Neither (Positive, neutral)
    "5) I am so sick of the traffic today, it's ridiculous! 🚗 (Offensive)",  # Offensive Language (Expressing frustration with traffic, negative tone but not targeting specific people or using hate speech)
    "6) You are such a worthless piece of garbage! 🤬 (Hate)",  # Hate speech
    "7) I don't understand why people are still so stupid about climate change! 🌍 (Offensive)",  # Offensive language
    "8) I absolutely love my new puppy, she's the cutest thing ever! 🐶 (Neither)",  # Neither (neutral)
    "9) People who are racist are disgusting, they should be ashamed of themselves. 😡 (Hate)",  # Hate speech
    "10) This restaurant has the worst food I've ever tasted, what a waste of money! 😤 (Offensive)",  # Offensive language
    "11) I'm so proud of what we've accomplished, it's an amazing feeling! 🏆 (Neither)",  # Neither (positive)
    "12) Why are all men so pathetic? It's honestly so embarrassing. 😒 (Hate)", # Hate speech
    "13) i hate jung ki soo much ", # Hate speech
    "14) you are such a disgusting person"
]

# Step 1: Preprocess the new tweets
# Use the same preprocessing function that was applied during training
new_cleaned_tweets = [text_preprocess(tweet) for tweet in new_tweets]

# Step 2: Transform the tweets using the trained TF-IDF vectorizer
new_tf_data = tf_idf.transform(new_cleaned_tweets)

# Step 3: Predict the class of the new tweets using the trained RandomForest model
new_predictions = ranfor_model.predict(new_tf_data)

# Print the results
for tweet, prediction in zip(new_tweets, new_predictions):
    print(f"Tweet: {tweet}")
    if prediction == 0:
        print(f"Predicted Class: Hate")
    elif prediction == 1:
        print(f"Predicted Class: Offensive Language")
    elif prediction == 2:
        print(f"Predicted Class: Neither")
    print("-------------------------------")


Tweet: 1) I hate this place, it's full of rude people! #frustrated (Offensive)
Predicted Class: Hate
-------------------------------
Tweet: 2) This is an amazing movie! I loved every second of it! 😍 (Neither)
Predicted Class: Neither
-------------------------------
Tweet: 3) Why is everyone so mean on the internet these days? 😡 (Offensive)
Predicted Class: Offensive Language
-------------------------------
Tweet: 4) I love learning new things, it's such a great experience. (Neither)
Predicted Class: Neither
-------------------------------
Tweet: 5) I am so sick of the traffic today, it's ridiculous! 🚗 (Offensive)
Predicted Class: Neither
-------------------------------
Tweet: 6) You are such a worthless piece of garbage! 🤬 (Hate)
Predicted Class: Hate
-------------------------------
Tweet: 7) I don't understand why people are still so stupid about climate change! 🌍 (Offensive)
Predicted Class: Offensive Language
-------------------------------
Tweet: 8) I absolutely love my new puppy, 

## Training with some other Models

In [208]:
# Train Random Forest model
ranfor_model = RandomForestClassifier()
ranfor_model.fit(x_train_balanced, y_train_balanced)

# Evaluate the model
y_pred2 = ranfor_model.predict(tf_test_data)
print("Accuracy:", accuracy_score(y_test, y_pred2))
print("Classification Report:")
print(classification_report(y_test, y_pred2))

Accuracy: 0.9477272727272728
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1217
           1       0.95      0.94      0.94      2009
           2       0.94      0.94      0.94      1174

    accuracy                           0.95      4400
   macro avg       0.95      0.95      0.95      4400
weighted avg       0.95      0.95      0.95      4400



In [ ]:
# tf-idf (12000) gives 88% Accuracy, while tf-idf(15000) gives 89% accuracy

In [27]:
# Train Random Forest model
ranfor_model = RandomForestClassifier()
ranfor_model.fit(x_train_balanced, y_train_balanced)

# Evaluate the model
y_pred2 = ranfor_model.predict(tf_test_data)
print("Accuracy:", accuracy_score(y_test, y_pred2))
print("Classification Report:")
print(classification_report(y_test, y_pred2))

Accuracy: 0.8833972160580996
Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.32      0.38       290
           1       0.93      0.93      0.93      3832
           2       0.79      0.84      0.82       835

    accuracy                           0.88      4957
   macro avg       0.72      0.70      0.71      4957
weighted avg       0.88      0.88      0.88      4957



## Hyperparameter Tunning

In [34]:
# tuning parameters

print("Starting Training Models...")

print("..." * 40)


# Initialize models for comparison
models = {
#     "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=300, C=2.0, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(),
#     "Random Forest": RandomForestClassifier(n_estimators=200, max_depth=40, random_state=42, class_weight='balanced'),
#     "SVC": SVC(C=1.5, kernel='linear', class_weight='balanced')
}

# Train and evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(x_train_balanced, y_train_balanced)
    
    # Predict and evaluate
    y_pred = model.predict(tf_test_data)
    print(f"\n\nModel: {model_name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("\n" + "-"*50 + "\n")


Starting Training Models...
........................................................................................................................


Model: Logistic Regression
Accuracy: 0.8583820859390761
Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.49      0.38       290
           1       0.95      0.88      0.91      3832
           2       0.77      0.89      0.82       835

    accuracy                           0.86      4957
   macro avg       0.68      0.75      0.71      4957
weighted avg       0.88      0.86      0.87      4957


--------------------------------------------------



Model: Random Forest
Accuracy: 0.8882388541456526
Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.33      0.40       290
           1       0.93      0.94      0.93      3832
           2       0.81      0.84      0.82       835

    accuracy                           0.89

In [40]:
# Train Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(x_train_balanced, y_train_balanced)


# Evaluate the model
y_pred = nb_model.predict(tf_test_data)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7859592495460964
Classification Report:
              precision    recall  f1-score   support

           0       0.21      0.56      0.31       290
           1       0.93      0.81      0.87      3832
           2       0.73      0.75      0.74       835

    accuracy                           0.79      4957
   macro avg       0.62      0.71      0.64      4957
weighted avg       0.86      0.79      0.81      4957



In [41]:
# Train Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(x_train_balanced, y_train_balanced)

# Evaluate the model
y_pred1 = nb_model.predict(tf_test_data)
print("Accuracy:", accuracy_score(y_test, y_pred1))
print("Classification Report:")
print(classification_report(y_test, y_pred1))

Accuracy: 0.7859592495460964
Classification Report:
              precision    recall  f1-score   support

           0       0.21      0.56      0.31       290
           1       0.93      0.81      0.87      3832
           2       0.73      0.75      0.74       835

    accuracy                           0.79      4957
   macro avg       0.62      0.71      0.64      4957
weighted avg       0.86      0.79      0.81      4957



In [43]:
# Train SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(x_train_balanced, y_train_balanced)

# Evaluate the model
y_pred3 = svm_model.predict(tf_test_data)
print("Accuracy:", accuracy_score(y_test, y_pred3))
print("Classification Report:")
print(classification_report(y_test, y_pred3))

Accuracy: 0.8739156748033085
Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.43      0.37       290
           1       0.95      0.90      0.92      3832
           2       0.82      0.89      0.85       835

    accuracy                           0.87      4957
   macro avg       0.70      0.74      0.72      4957
weighted avg       0.89      0.87      0.88      4957

